In [1]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.dirname(os.getcwd()))
import tensorflow as tf
import numpy as np
from waymo_open_dataset import dataset_pb2 as open_dataset
from utils import lidar_crop, save_darknet, label_in_polygon

2022-07-22 14:04:34.701067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-07-22 14:04:34.701084: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
TOO_DARK_THRESHOLD = 50
SCALED_WIDTH = 800
SCALED_HEIGHT = 533
LABEL_VEHICLE = 1
LABEL_PEDESTRIAN = 2
LABEL_CYCLIST = 4
SUPPORTED_LABELS = {
    LABEL_VEHICLE: 0,
    LABEL_PEDESTRIAN: 1,
    LABEL_CYCLIST: 2
}

In [4]:
training_files = ['Data/unprocessed/training/' + file for file in os.listdir('Data/unprocessed/training')]
dataset = tf.data.TFRecordDataset(training_files, buffer_size=tf.constant(int(pow(10,6)), tf.int64), num_parallel_reads=16)
OUTPUT_PATH = 'Data/darknet-data2'
img_id = len(os.listdir(f'{OUTPUT_PATH}/obj')) // 2

training_index = open('Data/darknet-data2/training.txt', 'w')
print('Processing training data')
nb_pedestrians = 0
MAX_PEDESTRIANS = 20000
nb_frames_processed = 0
for data in dataset:
    nb_frames_processed += 1

    if nb_frames_processed % 100 == 0:
        print('Frames processed: ', nb_frames_processed)
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))

    labels = [camera_labels.labels for camera_labels in frame.camera_labels if camera_labels.name == 1][0]
    labels = [label for label in labels if label.box.length > 50 and label.box.length > 50 and label.type in SUPPORTED_LABELS.keys()]
    label_types = [label.type for label in labels]

    if LABEL_PEDESTRIAN in label_types:
        nb_pedestrians += 1

    if (nb_pedestrians <= MAX_PEDESTRIANS or LABEL_PEDESTRIAN not in label_types) and LABEL_CYCLIST not in label_types:
        continue

    image = frame.images[0]
    image = np.array(tf.io.decode_jpeg(image.image))

    if np.mean(image) < TOO_DARK_THRESHOLD:
        continue
    
    og_width = image.shape[1]
    og_height = image.shape[0]
    
    save_darknet(image, labels, OUTPUT_PATH, og_width, og_height, training_index, img_id)
    
    img_id += 1
    if img_id % 100 == 0:
        print('nb_images: ', img_id)

training_index.close()

2022-07-21 23:13:05.196115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 23:13:05.196390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/antoinebrassardlahey/anaconda3/envs/mti830-3d/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2022-07-21 23:13:05.196471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/antoinebrassardlahey/anaconda3/envs/mti830-3d/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2022-07-21 23:13:05.196528: W tensorflow/stream_ex

Processing training data
Frames processed:  100
Frames processed:  200
Frames processed:  300
Frames processed:  400
Frames processed:  500
Frames processed:  600
Frames processed:  700
Frames processed:  800
Frames processed:  900
Frames processed:  1000
Frames processed:  1100
Frames processed:  1200
Frames processed:  1300
Frames processed:  1400
Frames processed:  1500
Frames processed:  1600
Frames processed:  1700
Frames processed:  1800
Frames processed:  1900
Frames processed:  2000
Frames processed:  2100
Frames processed:  2200
Frames processed:  2300
Frames processed:  2400
Frames processed:  2500
Frames processed:  2600
Frames processed:  2700
Frames processed:  2800
Frames processed:  2900
Frames processed:  3000
Frames processed:  3100
Frames processed:  3200
Frames processed:  3300
Frames processed:  3400
Frames processed:  3500
Frames processed:  3600
Frames processed:  3700
Frames processed:  3800
Frames processed:  3900
Frames processed:  4000
Frames processed:  4100


In [5]:
validation_files = ['Data/unprocessed/validation/' + file for file in os.listdir('Data/unprocessed/validation')]
dataset = tf.data.TFRecordDataset(validation_files, buffer_size=tf.constant(int(pow(10,6)), tf.int64), num_parallel_reads=16)
OUTPUT_PATH = 'Data/darknet-data2'
img_id = len(os.listdir(f'{OUTPUT_PATH}/obj')) // 2

validation_index = open('Data/darknet-data2/validation.txt', 'w')
print('Processing validation data')
nb_pedestrians = 0
MAX_PEDESTRIANS = 20000
nb_frames_processed = 0
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    nb_frames_processed += 1
    if nb_frames_processed % 100 == 0:
        print('Frames processed: ', nb_frames_processed)

    labels = [camera_labels.labels for camera_labels in frame.camera_labels if camera_labels.name == 1][0]
    labels = [label for label in labels if label.box.length > 50 and label.box.length > 50 and label.type in SUPPORTED_LABELS.keys()]
    label_types = [label.type for label in labels]

    if (nb_pedestrians <= MAX_PEDESTRIANS or LABEL_PEDESTRIAN not in label_types) and LABEL_CYCLIST not in label_types:
        continue

    image = frame.images[0]
    image = np.array(tf.io.decode_jpeg(image.image))

    if np.mean(image) < TOO_DARK_THRESHOLD:
        continue
    
    og_width = image.shape[1]
    og_height = image.shape[0]
    
    save_darknet(image, labels, OUTPUT_PATH, og_width, og_height, validation_index, img_id)
    
    img_id += 1
    if img_id % 100 == 0:
        print('nb_images: ', img_id)

validation_index.close()

Processing validation data
Frames processed:  100
Frames processed:  200
Frames processed:  300
Frames processed:  400
Frames processed:  500
Frames processed:  600
Frames processed:  700
Frames processed:  800
Frames processed:  900
Frames processed:  1000
Frames processed:  1100
Frames processed:  1200
Frames processed:  1300
Frames processed:  1400
Frames processed:  1500
Frames processed:  1600
Frames processed:  1700
Frames processed:  1800
nb_images:  20500
Frames processed:  1900
Frames processed:  2000
Frames processed:  2100
Frames processed:  2200
Frames processed:  2300
Frames processed:  2400
Frames processed:  2500
Frames processed:  2600
Frames processed:  2700
Frames processed:  2800
Frames processed:  2900
Frames processed:  3000
Frames processed:  3100
Frames processed:  3200
Frames processed:  3300
Frames processed:  3400
Frames processed:  3500
Frames processed:  3600
Frames processed:  3700
Frames processed:  3800
Frames processed:  3900
Frames processed:  4000
Fram